## OpenAI Fine-Tuning

In [4]:
import os
import getpass
from langsmith import Client
import datetime
from langchain import chains, chat_models, prompts, schema, callbacks
from langchain_core.output_parsers.string import StrOutputParser

In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [8]:
client = Client()

#### List all LLM runs for a specific project.

In [153]:
project_name = "default"
run_type = "llm"
end_time = datetime.datetime.now()

runs = client.list_runs(
    project_name=project_name,
    run_type=run_type,
    error=False,
)

#### Filter by feedback

Depending on how you're fine-tuning, you'll likely want to filter out 'bad' examples (and want to filter in 'good' examples).

You can directly list by feedback! Usually you assign feedback to the root of the run trace, so we will use 2 queries.


In [128]:
chain = (
    prompts.ChatPromptTemplate.from_template("Tell a joke for:\n{input}")
    | chat_models.ChatOpenAI(model="gpt-3.5-turbo-16k", tags=["my-chatgpt-run"])
    | StrOutputParser()
)

with callbacks.collect_runs() as cb:
    chain.invoke({"input": "foo"})
    # Assume feedback is logged
    run = cb.traced_runs[0]
    client.create_feedback(run.id, key="user_click", score=1)

In [129]:
project_name = "default"
end_time = datetime.datetime.now()

runs = client.list_runs(
    project_name=project_name,
    execution_order=1,
    filter='and(eq(feedback_key, "user_click"), eq(feedback_score, 1))',
    # For continuous scores, you can filter for >, <, >=, <= with the followingg arguments: gt/lt/gte/lte(feedback_score, 0.9)
    # filter='and(eq(feedback_key, "user_click"), gt(feedback_score, 0.9))',
    error=False,
)

In [130]:
llm_runs = []
for run in runs:
    llm_run = next(
        client.list_runs(
            project_name=project_name, run_type="llm", parent_run_id=run.id
        )
    )
    llm_runs.append(llm_run)

llm_runs[0].tags

['my-cool-llm-tag']

#### Filter by tags

In [132]:
# For any "Chain" object, you can add tags directly on the Example with LLMChain
import uuid

unique_tag = "call:fixed-uuid-for-testing"

chain = chains.LLMChain(
    llm=chat_models.ChatOpenAI(model="gpt-3.5-turbo-16k", 
        tags=["my-cool-llm-tag"]
    ),  # This tag will only be applied to the LLM
    prompt=prompts.ChatPromptTemplate.from_template(
        "Tell a joke based on the following prompt:\n\nPrompt:{input}"
    ),
    tags=["my-tag"],
)

# You can also define at call time for the call/invoke/batch methods.
# This tag will be propagated to all child calls
print(chain({"input": "podcasting these days"}, tags=[unique_tag]))

# If you're defining using Runnables (aka langchain expression language)
runnable = (
    prompts.ChatPromptTemplate.from_template(
        "Tell a joke based on the following prompt:\n\nPrompt:{input}"
    )
    | chat_models.ChatOpenAI(model="gpt-3.5-turbo-16k", 
        tags=["my-cool-llm-tag"]
    )  # This tag will only be applied to the LLM
    | schema.StrOutputParser(tags=["some-parser-tag"])
)

# Again, you can tag at call time as well. This tag will be propagated to all child calls
print(runnable.invoke({"input": "podcasting these days"}, {"tags": [unique_tag]}))

{'input': 'podcasting these days', 'text': 'Why did the podcaster bring a ladder to the recording studio?\n\nBecause they wanted to reach new heights in the podcasting industry!'}
Why did the podcast go on a diet?

Because it was tired of being a heavy "audio" file!


In [ ]:
chat_model.

In [147]:
project_name = "default"
end_time = datetime.datetime.now()

runs = client.list_runs(
    project_name=project_name,
    execution_order=1,  # Only return the root trace
    filter=f'has(tags, "{unique_tag}")',
)
run_list = list(runs)

# Print the number of runs
print(f"Number of runs: {len(run_list)}")

# Print details of each run
for run in run_list:
    print(run)
    print("_______________________")

Number of runs: 4
id=UUID('5045abc7-36f4-4075-84c8-c1fa788b9b52') name='RunnableSequence' start_time=datetime.datetime(2024, 7, 14, 23, 24, 47, 826458) run_type='chain' end_time=datetime.datetime(2024, 7, 14, 23, 24, 48, 396538) extra={'runtime': {'sdk': 'langsmith-py', 'sdk_version': '0.1.83', 'library': 'langsmith', 'platform': 'macOS-13.6.6-arm64-arm-64bit', 'runtime': 'python', 'py_implementation': 'CPython', 'runtime_version': '3.10.0', 'langchain_version': '0.2.7', 'langchain_core_version': '0.2.17', 'thread_count': 25.0, 'mem': {'rss': 77987840.0}, 'cpu': {'time': {'sys': 6.513377792, 'user': 24.075692032}, 'ctx_switches': {'voluntary': 197652.0, 'involuntary': 0.0}, 'percent': 0.0}}, 'metadata': {}} error=None serialized=None events=[{'name': 'start', 'time': '2024-07-14T23:24:47.826458+00:00'}, {'name': 'end', 'time': '2024-07-14T23:24:48.396538+00:00'}] inputs={'input': 'podcasting these days'} outputs={'output': 'Why did the podcast go on a diet?\n\nBecause it was tired of b

#### Filter by run name

In [54]:
project_name = "default"
run_type = "llm"
end_time = datetime.datetime.now()

runs = client.list_runs(
    project_name=project_name,
    run_type=run_type,
    filter='eq(name, "ChatOpenAI")',
    error=False,
)

In [52]:
print(runs)

<generator object Client.list_runs at 0x10cdc1af0>


#### Retrieve input prompts directly

In [10]:
# Example chain for the following query
from langchain import prompts, chat_models

chain = (
    prompts.ChatPromptTemplate.from_template(
        "Summarize the following chat log: {input}"
    )
    | chat_models.ChatOpenAI()
)

chain.invoke({"input": "hi there, hello...."})

/Users/sandragross/miniconda3/envs/chatbot-langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='A simple exchange of greetings between two people.', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 20, 'total_tokens': 29}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bf247ad1-d37a-4b04-9dfa-de72263c1093-0')

In [11]:
project_name = "default"
run_type = "prompt"
end_time = datetime.datetime.now()

runs = client.list_runs(
    project_name=project_name,
    run_type = run_type, 
    #end_time=end_time,
    error=False,
)

run_list = list(runs)

# Print the number of runs
print(f"Number of runs: {len(run_list)}")

# Print details of each run
for run in run_list:
    print(run.inputs)
    print(run.outputs)

Number of runs: 50
{'input': 'hi there, hello....'}
{'output': {'messages': [{'content': 'Summarize the following chat log: hi there, hello....', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'example': False}]}}
{'input': 'podcasting these days'}
{'output': {'messages': [{'content': 'Tell a joke based on the following prompt:\n\nPrompt:podcasting these days', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'example': False}]}}
{'input': 'foo'}
{'output': {'messages': [{'content': 'Tell a joke for:\nfoo', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'example': False}]}}
{'input': 'foo'}
{'output': {'messages': [{'content': 'Tell a joke for:\nfoo', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'example': False}]}}
{'input': 'foo'}
{'output': {'messages': [{'content': 'Tell a joke for:\nfoo', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'example': False}]}}
{'input': 'foo'}
{'output':

In [17]:
# You can then get a sibling LLM run by searching by parent_run_id and including other criteria
for prompt_run in runs:
    print("hi")
    llm_run = next(
        client.list_runs(
            project_name=project_name,
            run_type="llm",
            parent_run_id=prompt_run.parent_run_id,
        )
    )
    inputs, outputs = prompt_run.inputs, llm_run.outputs
    print("hi")
    print(inputs)
    print(outputs)

#### Add to a training dataset

In [178]:
dataset = client.create_dataset(
    dataset_name="Fine-Tuning Dataset Example 6",
    description=f"Chat logs taken from project {project_name} for fine-tuning",
    data_type="chat",
)
for run in runs:
    print(run)
    if "messages" not in run.inputs or not run.outputs:
        # Filter out non chat runs
        print(fail)
        continue
    try:
        # Convenience method for creating a chat example
        client.create_example_from_run(
            dataset_id=dataset.id,
            run=run,
        )
        # Or if you want to select certain keys/values in inputs
        # inputs = convert_inputs(run.inputs)
        # outputs = convert_outputs(run.outputs)
        # client.create_example(
        #     dataset_id=dataset.id,
        #     inputs=inputs,
        #     outputs=outputs,
        #     run=run,
        # )
    except:
        # Duplicate inputs raise an exception
        pass

#### Load examples as messages

In [9]:
for example in client.list_examples(dataset_name="Fine-Tuning Dataset Example"):
    print(example)


dataset_id=UUID('742051fb-adf6-49bc-b208-27e6374c3cf6') inputs={'input': [{'data': {'type': 'system', 'content': 'You are a helpful assistant.'}, 'type': 'system'}, {'data': {'type': 'human', 'content': 'What is LangChain?'}, 'type': 'human'}]} outputs={'output': {'data': {'id': 'run-5352e44f-a1e4-4f63-b1d4-cc95dc6effbc', 'type': 'AIMessageChunk', 'content': 'I\'m sorry, but I couldn\'t find any information about "LangChain". Could you please provide more context or clarify your question?', 'tool_calls': [], 'response_metadata': {'model_name': 'gpt-3.5-turbo-16k-0613', 'finish_reason': 'stop'}, 'invalid_tool_calls': []}, 'type': 'aichunk'}} metadata={'dataset_split': ['base']} id=UUID('fd9a01a2-0ee8-4ecf-a063-ea703f2ae7d6') created_at=datetime.datetime(2024, 7, 14, 23, 43, 14, 167796, tzinfo=datetime.timezone.utc) modified_at=datetime.datetime(2024, 7, 14, 23, 43, 14, 167796, tzinfo=datetime.timezone.utc) runs=[] source_run_id=None
dataset_id=UUID('742051fb-adf6-49bc-b208-27e6374c3cf6'

In [19]:
def convert_to_message_format(examples):
    message_data = []

    for example in examples:
        # Extract input messages
        input_messages = example["inputs"]["input"]
        # Extract output message
        output_message = example["outputs"]["output"]["data"]["content"]

        # Initialize the message list
        messages = []

        # Add system and user messages from inputs
        for msg in input_messages:
            role = "system" if msg["data"]["type"] == "system" else "user"
            content = msg["data"]["content"]
            messages.append({"role": role, "content": content})

        # Add assistant message from output
        messages.append({"role": "assistant", "content": output_message})

        # Append the structured message to the data list
        message_data.append({"messages": messages})

    return message_data

In [21]:
messages = [
    convert_to_message_format(example)
    for example in client.list_examples(dataset_name="Fine-Tuning Dataset Example")
]

TypeError: tuple indices must be integers or slices, not str

In [ ]:
messages